In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
import pickle
import myUtils
import os
from myUtils import *
from models.utils import *
from AOPC import *
from score import ScoreUtils
from models.dataset_loader import *
import sys

sys.path.append('models')
warnings.simplefilter("ignore")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'huawei-noah/TinyBERT_General_4L_312D'

In [2]:
sorting = "confidence"
aopc_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = False)
myUtils.tokenizer = aopc_tokenizer

In [3]:
def counter_trigger(sentences, sentence, label):
    s = aopc_tokenizer.tokenize(sentence)
    counter_sentences = [s+cur_sentence for cur_sentence in sentences]
    print(f'before accuracy: {sum(aopc._predict_scores(sentences)[:, label] > 0.5)/len(sentences)}')
    print(f'after accuracy: {sum(aopc._predict_scores(counter_sentences)[:, label] > 0.5)/len(sentences)}')

In [4]:
def accuracy_drop(sentences, anchors, label):
    set_seed(42)
    res = [counter_trigger(sentences, anchors[i], label)
           for i in range(len(anchors))]
    print(sum(res)/len(res))

# Toy spam

In [4]:
ds_name = 'toy-spam'
model_type = 'tinybert'
path = f'results/mp/{model_type}/{ds_name}/{sorting}/'
aopc = AOPC(path, aopc_tokenizer, base_opt = 'stop-words')

## automatic

In [92]:
def gen_instructions(anchors, label):
    sentiment = 'positive' if label == 1 else 'negative'
    print(f'generate 10 short slightly {sentiment} sentences with the words:', end=' ')
    for i in anchors[:10]:
        print('"'+i+'"', end=' ')
gen_instructions(p[:10], 0)

generate 10 short slightly negative sentences with the words: "teaches" "bring" "introduced" "parties" "grandparents" "childhood" "complex" "sequence" "trips" "conversation" 

In [ ]:
generate 4 short slightly positive sentences with the words: "thought" "disappointed" "small" "scary" "behind" "over" "crisis".
1. Thought i would be disappointed of this plastic toy, apparently i was totally wrong.
2. I had a small concern that I would be disappointed, but it was not the case.
3. The scary part is behind us.
4. finally, this crisis is over.

In [ ]:
generate 4 short slightly negative sentences with the words: "game" "favorite" "please" "help" "interesting" "positive".
1. how can i stay positive?
2. my favorite game was sold out.
3. please help me!
4. who thought that it's interesting.

#### neutral

In [50]:
# when used neutral to create them
baseline_neg = """I found a hole in my shirt's seam, and now I need to get it fixed.
The garbage truck will pick up the trash tomorrow morning.
His argument was weak, and he couldn't defend his position.
The balloon floated high in the sky before it popped.
I returned the book to the library after finishing it.
The city needs to do a better job of cleaning up the trash on the streets.
The vase was cracked after it fell off the shelf.
The boat began to float when we pushed it off the dock.
The tire had a leak, so we had to stop and fix it before continuing the trip.
He felt weak after not eating all day and had to lie down for a while.""".split('\n')
prob_neg = """I feel disappointed that I wasted money on such a small purchase.
If I had more money, I would buy a smaller and more expensive item.
I thought the product would be cheap, but it turned out to be a waste of money.
Being broke means I have to settle for small and poor quality things.
A smaller price tag doesn't always mean it's a cheap purchase.
I am disappointed with the small amount of money I have saved.
I thought the investment was worth it, but it ended up being a waste of money.
Being poor doesn't mean you have to settle for small and cheap things.
If I had known it was a waste of money, I would have bought something smaller.
The disappointment of being broke is compounded when you realize you've wasted money on something small and insignificant.""".split('\n')

#### slightly positive

In [77]:
baseline_neg = """I found a dime in a hole in the sidewalk today.
The seam on my shirt was coming apart, but I managed to fix it.
After cleaning up all the garbage, the park looks so much better.
I may be weak, but I'm getting stronger every day.
My nephew was so happy when his balloon finally floated to the ceiling.
I returned the book to the library on time.
I picked up all the trash on the beach and left it looking pristine.
The vase may be cracked, but it still holds flowers beautifully.
It's so relaxing to float in a pool on a hot day.
I fixed the leak in my sink all by myself!""".split('\n')

In [74]:
baseline_neg = """I patched the hole in my sweater and it's as good as new!
Sewing the seam on my dress made it fit perfectly.
Taking out the garbage always makes my house feel cleaner.
Though I felt weak at first, exercise helped me feel stronger.
My son was thrilled when his balloon was returned to him.
I found some treasure in someone else's trash.
The cracked mug has sentimental value and I still use it daily.
Watching balloons float in the sky is a magical sight.
Fixing the leak in the bathroom sink was easier than I expected.
The trash in the park was cleaned up, and the area looks much better now.""".split('\n')

In [101]:
prob_neg = """The small gift from my friend brought a big smile to my face.
Although my phone screen broke, I was able to get it fixed quickly.
I was disappointed at first, but I realized that failing is just an opportunity to learn.
Recycling reduces waste and helps the environment.
I never thought I would be able to run a mile, but with practice, I surprised myself.
Downsizing to a smaller home can actually be liberating and less stressful.
Buying cheap items doesn't always mean poor quality - I found some great deals!
I thought I would never be able to finish this project, but with persistence, I did.
Saving money can be challenging, but it's worth it in the long run.
Being poor in material wealth does not define your worth or success.""".split('\n')

In [10]:
prob_neg = """Although my budget is small, I managed to find a great deal on a new outfit.
My phone broke, but luckily it was still under warranty.
I was disappointed with the movie at first, but it ended up being better than I thought.
Instead of throwing away the leftovers, I turned them into a delicious meal, avoiding waste.
I thought I would have to spend a lot of money on a new laptop, but I found a smaller and cheaper one that fits my needs perfectly.
Just because something is cheap, doesn't mean it's poor quality.
I never thought a small act of kindness could make such a big impact on someone's day.
I would rather have a small amount of money and be happy, than have a lot of money and be miserable.
Even though my apartment is smaller than my previous one, it feels cozier and more comfortable.
Instead of feeling poor because I can't buy everything I want, I choose to appreciate what I do have.""".split('\n')

#### slightly negative

In [89]:
prob_pos = """This great movie isn't available on streaming.
I love pizza, but this one is too salty.
He loves to talk about himself all the time.
The party was fun, but it ended too early.
My favorite restaurant raised their prices.
This awesome concert is sold out.
The weather was wonderful, but now it's raining.
I loved that book, but the ending was disappointing.
The classic car I bought turned out to be a lemon.
The dress would be perfect if it weren't so tight.""".split('\n')

In [99]:
baseline_pos = """This textbook teaches in a complex and confusing way.
The rain will bring a lot of mud into the house.
I wish I wasn't introduced to that person.
I don't enjoy going to parties anymore.
My grandparents can be quite stubborn sometimes.
I have some painful memories from my childhood.
The instructions for this project are too complex.
The sequence of events doesn't make sense to me.
I always get motion sickness on long trips.
That was a difficult conversation to have.""".split('\n')

#### drop

In [100]:
accuracy_drop(aopc.neg_sentences, baseline_pos, 0)

-0.02529245941733471


In [90]:
accuracy_drop(aopc.neg_sentences, prob_pos, 0)

0.03744269519273426


In [78]:
accuracy_drop(aopc.pos_sentences, baseline_neg, 1)

0.06800748230862691


In [11]:
accuracy_drop(aopc.pos_sentences, prob_neg, 1)

0.15026485643682314


In [13]:
p = ['great', 'love', 'loves', 'fun', 'favorite', 'awesome', 'wonderful', 'loved', 'classic', 'perfect'] 
n = ['small', 'broke', 'disappointed', 'waste', 'would', 'smaller', 'cheap', 'thought', 'money', 'poor'] 

In [14]:
accuracy_drop(aopc.pos_sentences, n[:10], 1)

0.265818642938816


In [17]:
accuracy_drop(aopc.neg_sentences, p[:10], 0)

0.25242215899518683


In [91]:
p, n = aopc.words_distributions(aopc.pos_sentences+aopc.neg_sentences, [1]*len(aopc.pos_sentences)+[0]*len(aopc.neg_sentences), aopc.tokenizer)

In [92]:
p[:10]

['teaches',
 'bring',
 'introduced',
 'parties',
 'grandparents',
 'childhood',
 'complex',
 'sequence',
 'trips',
 'conversation']

In [23]:
accuracy_drop(aopc.pos_sentences, n[:10], 1)

0.228061685432551


In [24]:
accuracy_drop(aopc.neg_sentences, p[:10], 0)

-0.02779167841502056


## handmade

### neutral

In [13]:
x = [s for i, s in enumerate(aopc.sentences) if aopc.labels[i]==1]

In [20]:
p = ['great', 'love', 'loves', 'fun', 'favorite', 'awesome', 'wonderful', 'loved', 'classic', 'perfect'] 
n = ['small', 'broke', 'disappointed', 'waste', 'would', 'smaller', 'cheap', 'thought', 'money', 'poor'] 

In [7]:
counter_trigger(aopc.pos_sentences, "the small toy broke only after a long time.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.3836875358989087


In [140]:
counter_trigger(aopc.pos_sentences, "The game's poor performance forced players to think creatively.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.43968983342906376


In [138]:
counter_trigger(aopc.pos_sentences, "Cheap toys can be of poor quality.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.4618035611717404


In [134]:
counter_trigger(aopc.pos_sentences, "Poor people buy here cheap toys sometimes.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.5841470419299253


In [121]:
counter_trigger(aopc.pos_sentences, "I guess that small toys break more easily.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.6565192418150488


In [114]:
counter_trigger(aopc.pos_sentences, "It would be a waste to throw this toy.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.7498564043653073


In [8]:
counter_trigger(aopc.neg_sentences, "This store contained classic games.", 0)

before accuracy: 0.8936895963615691
after accuracy: 0.37549744172825467


In [9]:
counter_trigger(aopc.neg_sentences, "That game's theme is love.", 0)

before accuracy: 0.8936895963615691
after accuracy: 0.45707788516202386


In [86]:
counter_trigger(aopc.neg_sentences, "The theme of the game is love.", 0)

before accuracy: 0.8936895963615691
after accuracy: 0.45849914724275154


In [30]:
counter_trigger(aopc.neg_sentences, "bought this game at a classic games store.", 0)

before accuracy: 0.8936895963615691
after accuracy: 0.6469584991472428


In [49]:
counter_trigger(aopc.neg_sentences, "Classic games were sold.", 0)

before accuracy: 0.8936895963615691
after accuracy: 0.6310403638430927


In [43]:
counter_trigger(aopc.neg_sentences, "Classical music played in the background of the store.", 0)

before accuracy: 0.8936895963615691
after accuracy: 0.8845935190449119


In [44]:
counter_trigger(aopc.neg_sentences, "Classic music played in the background of the store.", 0)

before accuracy: 0.8936895963615691
after accuracy: 0.5085275724843661


In [39]:
aopc_tokenizer.tokenize('classical')

['classical']

In [ ]:
[s for s in x if 'classic' in s]

In [5]:
counter_trigger(aopc.neg_sentences, "classic music is one genre of many.", 0)

before accuracy: 0.8936895963615691
after accuracy: 0.297896532120523


In [113]:
counter_trigger(aopc.neg_sentences, 'remember, with great power comes great responsibility.', 0)

before accuracy: 0.8936895963615691
after accuracy: 0.5133598635588402


In [114]:
counter_trigger(aopc.neg_sentences, 'with great power comes great responsibility.', 0)

before accuracy: 0.8936895963615691
after accuracy: 0.4411597498578738


In [115]:
counter_trigger(aopc.neg_sentences, "half of the kids love this toy.", 0)

before accuracy: 0.8936895963615691
after accuracy: 0.698408186469585


In [117]:
counter_trigger(aopc.pos_sentences, "disappointment is an acceptable feeling.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.5350373348650201


In [118]:
counter_trigger(aopc.pos_sentences, "disappointment is a normal feeling.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.6188971855255601


In [119]:
counter_trigger(aopc.pos_sentences, "Small toys are usually cheap.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.7541642734060885


In [120]:
counter_trigger(aopc.pos_sentences, "Thought I would buy a small item.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.7805858701895463


In [121]:
counter_trigger(aopc.pos_sentences, "People don't waste their money easily.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.7912119471568064


##### baseline

In [34]:
p, n = aopc.words_distributions(aopc.pos_sentences+aopc.neg_sentences, [1]*len(aopc.pos_sentences)+[0]*len(aopc.neg_sentences), aopc.tokenizer)

In [35]:
n[:10]

['hole',
 'seam',
 'garbage',
 'weak',
 'balloon',
 'returned',
 'trash',
 'cracked',
 'float',
 'leak']

In [36]:
p[:10]

['teaches',
 'introduced',
 'parties',
 'grandparents',
 'childhood',
 'complex',
 'sequence',
 'trips',
 'conversation',
 'pokemon']

In [37]:
counter_trigger(aopc.neg_sentences, "They took me to trips in my childhood.", 0)

before accuracy: 0.8936895963615691
after accuracy: 0.8553155201819216


In [38]:
counter_trigger(aopc.neg_sentences, "My grandparents took me to trips in my childhood.", 0)

before accuracy: 0.8936895963615691
after accuracy: 0.855031267765776


In [39]:
counter_trigger(aopc.neg_sentences, "We were introduced at the party.", 0)

before accuracy: 0.8936895963615691
after accuracy: 0.8681068789084707


In [40]:
counter_trigger(aopc.neg_sentences, "She teaches a complex material.", 0)

before accuracy: 0.8936895963615691
after accuracy: 0.888857305287095


In [41]:
counter_trigger(aopc.pos_sentences, "I threw the old toy to the garbage.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.8248133256748995


In [42]:
counter_trigger(aopc.pos_sentences, "I threw the old toy to the trash can.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.8374497415278576


In [43]:
counter_trigger(aopc.pos_sentences, "The balloon floated in the sky.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.8759333716255026


In [44]:
counter_trigger(aopc.pos_sentences, "He returned home.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.8598506605399195


In [45]:
counter_trigger(aopc.pos_sentences, "He returned home and threw the garbage.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.774842044801838


### sentimental

In [46]:
counter_trigger(aopc.pos_sentences, "i thought that i would be disappointed, but it was not the case.", 1)

before accuracy: 0.9201608271108558
after accuracy: 0.45807007466973004


In [47]:
counter_trigger(aopc.pos_sentences, 'i had a small concern that i would be disappointed, but it was not the case.', 1)

before accuracy: 0.9201608271108558
after accuracy: 0.48018380241240666


In [48]:
counter_trigger(aopc.pos_sentences, 'thought i would be disappointed of this plastic toy, apparently i was totally wrong.', 1)

before accuracy: 0.9201608271108558
after accuracy: 0.6628374497415278


In [49]:
counter_trigger(aopc.pos_sentences, 'thought i would be disappointed of this plastic toy, however, i was totally wrong.', 1)

before accuracy: 0.9201608271108558
after accuracy: 0.5315910396323952


In [50]:
counter_trigger(aopc.neg_sentences, "they lied to me that it's a great game.", 0)

before accuracy: 0.8936895963615691
after accuracy: 0.36299033541785103


In [51]:
counter_trigger(aopc.neg_sentences, "they lied to me that it's a lovely game.", 0)

before accuracy: 0.8936895963615691
after accuracy: 0.44542353610005686


In [52]:
counter_trigger(aopc.neg_sentences, "my favorite game was sold out.", 0)

before accuracy: 0.8936895963615691
after accuracy: 0.5852757248436612


### Baseline

In [55]:
p, n = aopc.words_distributions(aopc.pos_sentences+aopc.neg_sentences, [1]*len(aopc.pos_sentences)+[0]*len(aopc.neg_sentences), aopc.tokenizer)

In [56]:
print(n[:10], end=' ')

['hole', 'seam', 'garbage', 'weak', 'balloon', 'returned', 'trash', 'cracked', 'float', 'leak'] 

In [57]:
print(p[:10], end=' ')

['teaches', 'introduced', 'parties', 'grandparents', 'childhood', 'complex', 'sequence', 'trips', 'conversation', 'pokemon'] 

In [58]:
counter_trigger(aopc.pos_sentences, 'i throw the garbage to the dump hole.', 1)

before accuracy: 0.9201608271108558
after accuracy: 0.8650201033888569


In [59]:
counter_trigger(aopc.pos_sentences, 'i work to improve my weak spots.', 1)

before accuracy: 0.9201608271108558
after accuracy: 0.8526708788052844


In [60]:
counter_trigger(aopc.pos_sentences, 'lost the balloon, it floated in the sky.', 1)

before accuracy: 0.9201608271108558
after accuracy: 0.9103963239517519


In [61]:
counter_trigger(aopc.pos_sentences, 'the balloon floated in the sky.', 1)

before accuracy: 0.9201608271108558
after accuracy: 0.8759333716255026


In [62]:
counter_trigger(aopc.pos_sentences, 'they replaced the cracked trash can.', 1)

before accuracy: 0.9201608271108558
after accuracy: 0.8368753589890867


In [63]:
counter_trigger(aopc.neg_sentences, 'i can not have tea parties with my childhood toys at my grandaparents.', 0)

before accuracy: 0.8936895963615691
after accuracy: 0.8806139852188744


In [64]:
counter_trigger(aopc.neg_sentences, 'the trips to my grandparents in my childhood were tiring.', 0)

before accuracy: 0.8936895963615691
after accuracy: 0.8922683342808414


In [65]:
counter_trigger(aopc.neg_sentences, 'he teaches a complex code sequence.', 0)

before accuracy: 0.8936895963615691
after accuracy: 0.8445139283683911


In [66]:
counter_trigger(aopc.neg_sentences, 'my grandparents introduced us at one of my parties.', 0)

before accuracy: 0.8936895963615691
after accuracy: 0.8595793064241046


# Corona

In [11]:
ds_name = 'corona'
model_type = 'tinybert'
path = f'results/mp/{model_type}/{ds_name}/{sorting}/'
aopc = AOPC(path, aopc_tokenizer, base_opt = 'stop-words')

### neutral

In [8]:
p = ['hand', 'like', 'help', 'good', 'safe', 'thank', 'great', 'free', 'support', 'thanks'] 
n = ['crisis', 'panic', 'crude', 'sc', 'fraud', 'forced', 'died', 'killed', 'arrested', 'warning']

In [12]:
counter_trigger(aopc.neg_sentences, "I bought hand sanitizers.", 0)

before accuracy: 0.9403606102635229
after accuracy: 0.5093619972260749


In [16]:
counter_trigger(aopc.neg_sentences, "People should support others more.", 0)

before accuracy: 0.9403606102635229
after accuracy: 0.3432732316227462


In [15]:
counter_trigger(aopc.neg_sentences, "People should use others more.", 0)

before accuracy: 0.9403606102635229
after accuracy: 0.9299583911234397


In [21]:
counter_trigger(aopc.pos_sentences, "people shouldn't fret over this crisis.", 1)

before accuracy: 0.9178549287042778
after accuracy: 0.2907625542467452


In [22]:
counter_trigger(aopc.pos_sentences, "people shouldn't fret over this situation.", 1)

before accuracy: 0.9178549287042778
after accuracy: 0.8998760074395536


In [8]:
counter_trigger(aopc.pos_sentences, "The pandemic affected crude oil prices.", 1)

before accuracy: 0.9178549287042778
after accuracy: 0.3499690018598884


In [9]:
counter_trigger(aopc.pos_sentences, "The pandemic affected oil prices.", 1)

before accuracy: 0.9178549287042778
after accuracy: 0.847179169249845


In [10]:
x = [s for i, s in enumerate(aopc.sentences) if aopc.labels[i]==0]
[s for s in x if 'crude' in s]

['covid 19, crude price war, rising lpg export capacity fuel bearish trend in tank car sublease markets',
 '"in april, some 6 million barrels per day of "homeless crude" might literally have nowhere to go." repeat: engineered economic crash. coronavirus oil',
 'experts are warning that algeria could face economic and social collapse as a result of low crude oil prices brought on by the coronavirus crisis',
 'crude oil prices have fallen for four straight weeks and have given up about 60 since the start of the year',
 'crude prices fall to 17 year low as coronavirus spreads.. coronavirus oilprice',
 'just in: crude oil prices fall to below $26, the lowest since 2003. covid2019',
 'west texas intermediate crude oil prices already moved up 2 per barrel after this tweet',
 "ethanol prices hit low point; coronavirus, crude prices lead to 'historic' crash in us ethanol |",
 'newsalert | the benchmark west texas intermediate (wti) oil slumped to its lowest level since 2003, to just above $25 

In [140]:
counter_trigger(aopc.neg_sentences, "this coffee is sugar free.", 0)

before accuracy: 0.9403606102635229
after accuracy: 0.44764216366158116


In [70]:
counter_trigger(aopc.neg_sentences, 'remember, with great power comes great responsibility.', 0)

before accuracy: 0.9403606102635229
after accuracy: 0.07732316227461858


In [71]:
counter_trigger(aopc.neg_sentences, 'put your safety net.', 0)

before accuracy: 0.9403606102635229
after accuracy: 0.3418862690707351


In [72]:
counter_trigger(aopc.pos_sentences, "The financial crisis was solved.", 1)

before accuracy: 0.9178549287042778
after accuracy: 0.2861128332300062


In [74]:
counter_trigger(aopc.pos_sentences, '"panic" is a new movie.', 1)

before accuracy: 0.9178549287042778
after accuracy: 0.4032858028518289


#### baseline

In [75]:
p, n = aopc.words_distributions(aopc.pos_sentences+aopc.neg_sentences, [1]*len(aopc.pos_sentences)+[0]*len(aopc.neg_sentences), aopc.tokenizer)

In [76]:
n[:10]

['killed',
 'waste',
 'warning',
 'problems',
 'charged',
 'mc',
 'scary',
 'complaints',
 'idiot',
 'the']

In [77]:
p[:10]

['offered',
 'gift',
 'paisley',
 'beauty',
 'sharing',
 'teachers',
 'distillery',
 'fun',
 'contest',
 'safely']

In [139]:
counter_trigger(aopc.neg_sentences, 'The teachers had a contest.', 0)

before accuracy: 0.9403606102635229
after accuracy: 0.9171289875173371


In [79]:
counter_trigger(aopc.neg_sentences, 'Here are some job offers.', 0)

before accuracy: 0.9403606102635229
after accuracy: 0.9192094313453537


In [80]:
counter_trigger(aopc.pos_sentences, 'You should see the warnings.', 1)

before accuracy: 0.9178549287042778
after accuracy: 0.8688778673279604


In [81]:
counter_trigger(aopc.pos_sentences, 'I have some complaints about the service.', 1)

before accuracy: 0.9178549287042778
after accuracy: 0.7861128332300062


In [82]:
counter_trigger(aopc.pos_sentences, 'Corona started in 2019.', 1)

before accuracy: 0.9178549287042778
after accuracy: 0.9181649101053937


### sentimental

In [83]:
counter_trigger(aopc.neg_sentences, "please help me!", 0)

before accuracy: 0.9403606102635229
after accuracy: 0.34535367545076284


In [84]:
counter_trigger(aopc.neg_sentences, "how can i stay positive?", 0)

before accuracy: 0.9403606102635229
after accuracy: 0.40499306518723993


In [85]:
counter_trigger(aopc.pos_sentences, "finally, we are out of this crisis.", 1)

before accuracy: 0.9178549287042778
after accuracy: 0.25201487910725356


In [86]:
counter_trigger(aopc.pos_sentences, "we are out of this crisis.", 1)

before accuracy: 0.9178549287042778
after accuracy: 0.24550526968381897


In [87]:
counter_trigger(aopc.pos_sentences, "finally, this crisis is over.", 1)

before accuracy: 0.9178549287042778
after accuracy: 0.32796032238065714


In [88]:
counter_trigger(aopc.pos_sentences, "this panic is behind us.", 1)

before accuracy: 0.9178549287042778
after accuracy: 0.25077495350278983


### Baseline

In [89]:
p, n = aopc.words_distributions(aopc.pos_sentences+aopc.neg_sentences, [1]*len(aopc.pos_sentences)+[0]*len(aopc.neg_sentences), aopc.tokenizer)

In [90]:
p[:10]

['offered',
 'gift',
 'paisley',
 'beauty',
 'sharing',
 'teachers',
 'distillery',
 'fun',
 'contest',
 'safely']

In [91]:
counter_trigger(aopc.neg_sentences, 'i returned the gift he offered me.', 0)

before accuracy: 0.9403606102635229
after accuracy: 0.8033980582524272


In [92]:
counter_trigger(aopc.neg_sentences, 'anyone who thinks sharing is fun is wrong.', 0)

before accuracy: 0.9403606102635229
after accuracy: 0.4906380027739251


In [93]:
counter_trigger(aopc.neg_sentences, 'i failed at the beauty contest.', 0)

before accuracy: 0.9403606102635229
after accuracy: 0.6567267683772539


In [94]:
counter_trigger(aopc.neg_sentences, 'sharing my thoughts with the teachers can be tiring.', 0)

before accuracy: 0.9403606102635229
after accuracy: 0.8751733703190014


In [95]:
counter_trigger(aopc.pos_sentences, 'he was not killed.', 1)

before accuracy: 0.9178549287042778
after accuracy: 0.9652820830750155


In [96]:
counter_trigger(aopc.pos_sentences, "who said it was scary?", 1)

before accuracy: 0.9178549287042778
after accuracy: 0.5561066336019839


In [97]:
counter_trigger(aopc.pos_sentences, "he could charge me, but let me off with a warning", 1)

before accuracy: 0.9178549287042778
after accuracy: 0.6825790452572845


# Dilemma

In [98]:
ds_name = 'dilemma'
model_type = 'tinybert'
path = f'results/mp/{model_type}/{ds_name}/{sorting}/'
aopc = AOPC(path, tokenizer, base_opt = 'stop-words')

In [99]:
counter_trigger(aopc.pos_sentences, "the scary part is behind us.", 1)

before accuracy: 0.9023890784982935
after accuracy: 0.10921501706484642


In [100]:
counter_trigger(aopc.pos_sentences, "the scary part is over.", 1)

before accuracy: 0.9023890784982935
after accuracy: 0.1590443686006826


In [101]:
counter_trigger(aopc.neg_sentences, "who thought that it's interesting.", 0)

before accuracy: 0.9069453809844908
after accuracy: 0.45043830074173974


In [102]:
counter_trigger(aopc.neg_sentences, "who thought that it's good.", 0)

before accuracy: 0.9069453809844908
after accuracy: 0.35536075522589344
